In [2]:
import folium
import pandas as pd
import json
import requests

In [38]:
projects= r'P3_GrantExport.csv'
state_data = pd.read_csv(projects, sep=';')
state_data.set_index(state_data.columns[0], inplace=True)
print(state_data.shape)
state_data = state_data[['University', 'Approved Amount']]
state_data.dropna(subset=['University'], inplace=True)
print(state_data.shape)
state_data = state_data[state_data['University'] != 'Nicht zuteilbar - NA'] #Nicht zuteilbar = NA University
state_data = state_data[state_data['University'] != 'NPO (Biblioth., Museen, Verwalt.) - NPO'] #NPO = is not university

(63969, 14)
(50988, 2)


In [73]:
def searchCantonGeoNames(university): #returns in which canton the university is in
    query  = university.split(" - ")
    url = 'http://api.geonames.org/searchJSON?country=CH&maxRows=1&username=jnls'
    payload = {'q': query[0] }
    req = requests.get(url, params=payload)
    req_json = json.loads(req.text)
    if req_json['totalResultsCount'] > 0:
        return req_json['geonames'][0]['adminCode1']
    elif req_json['totalResultsCount'] == 0 and len(query) > 1:
        payload = {'q': query[1] }
        req = requests.get(url, params=payload)
        req_json = json.loads(req.text)
        if req_json['totalResultsCount'] > 0:
            return req_json['geonames'][0]['adminCode1']
        else:
            return "NaN"
def search_cantons(data):
    UniCanton = data['University'].drop_duplicates().to_frame()
    UniCanton.set_index(UniCanton.University, inplace=True)
    UniCanton['Canton'] = ''
    for i in range(len(UniCanton)):
        canton = searchCantonGeoNames(UniCanton['University'].iloc[i])
        UniCanton['Canton'].iloc[i] = canton
    #data = data[data['Canton'] != 'NaN'] 
    return UniCanton

temp = search_cantons(state_data)

In [74]:
temp

,University,Canton
University,,
Université de Genève - GE,Université de Genève - GE,GE
Universität Basel - BS,Universität Basel - BS,BS
Université de Fribourg - FR,Université de Fribourg - FR,FR
Universität Zürich - ZH,Universität Zürich - ZH,ZH
Université de Lausanne - LA,Université de Lausanne - LA,VD
Universität Bern - BE,Universität Bern - BE,BE
"Eidg. Forschungsanstalt für Wald,Schnee,Land - WSL","Eidg. Forschungsanstalt für Wald,Schnee,Land -...",ZH
Université de Neuchâtel - NE,Université de Neuchâtel - NE,NE
ETH Zürich - ETHZ,ETH Zürich - ETHZ,ZH


In [31]:
cantonMap = pd.DataFrame({'Canton':['JU', 'ZH', 'BE', 'LU', 'SG', 'NE', 'VD', 'VS', 'GE', 'TI', 
                                 'SO', 'UR', 'OW', 'NW', 'SZ', 'GL', 'ZG', 'BL', 'BS', 'FR',
                                'SH', 'AR', 'AI', 'GR', 'AG', 'TG']})

ch_geo = r'ch-cantons.topojson.json'

map = folium.Map(location=[46.82244,8.22410], zoom_start=8)
map.choropleth(data=cantonMap, 
                columns=['Canton'], 
                geo_path=ch_geo,  
                fill_color='YlGn', fill_opacity=0.7, line_opacity=0.2,
                key_on='feature.id')
map.save('ch-cantons.html')

IndexError: list index out of range

In [33]:
url = 'http://api.geonames.org/searchJSON?country=CH&maxRows=1&username=jnls'
payload = {'q': 'EPFL'}
req = requests.get(url, params=payload)
req_json = json.loads(req.text)
print(req_json)

{'totalResultsCount': 3, 'geonames': [{'fcodeName': 'university', 'lat': '46.51939', 'adminName1': 'Vaud', 'countryId': '2658434', 'population': 0, 'toponymName': 'Ecole Polytechnique Fédérale de Lausanne', 'fclName': 'spot, building, farm', 'geonameId': 6285790, 'adminCode1': 'VD', 'name': 'Ecole Polytechnique Fédérale de Lausanne', 'fcl': 'S', 'countryCode': 'CH', 'countryName': 'Switzerland', 'fcode': 'UNIV', 'lng': '6.56673'}]}


In [41]:
"HOLA - ADIOS ".split("-")

['HOLA ', ' ADIOS ']